In [ ]:
import numpy as np
import types
import collections
from random import shuffle
import PIL
from PIL import Image
import cupy as cp
from timeit import default_timer as timer
from matplotlib.pyplot import imshow
import cv2
import os
from tqdm.notebook import trange, tqdm
%matplotlib inline

In [ ]:
file = "/tmp/test.jpeg"

In [ ]:
import torchvision.transforms as transforms
import torchvision.transforms.functional as fc
t1 = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip()
        ])
t2 = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()
        ])
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
t3 = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize
        ])


In [ ]:
img = Image.open(file)# 读取图像

In [ ]:
imshow(img)

In [ ]:
tensor1 = t1(img)

In [ ]:
tensor2 = t2(img)

In [ ]:
tensor3 = t3(img)

In [ ]:
tensor1

In [ ]:
tensor2

In [ ]:
tensor3

In [ ]:
import sys

In [ ]:
print("tensor1 size: {}".format(sys.getsizeof(tensor1.tobytes())))

In [ ]:
print("tensor2 size: {}".format(tensor2.element_size() * tensor2.nelement()))

In [ ]:
print("tensor3 size: {}".format(tensor2.element_size() * tensor2.nelement()))

In [ ]:
602112/150561.

In [ ]:
def showImageFromRaw(fname):
    with open(fname, 'r') as f:
        lines = f.readlines()
    height = int(lines[0])
    width = int(lines[1])
    print(height, width)
    raw = []
    for i in lines[2:]:
        sp = i.rstrip().split(" ")
        r,g,b = sp[0],sp[1],sp[2]
        raw.append([r,g,b])
    raw = np.asarray(raw, dtype=np.uint8)
    raw = raw.reshape(height, width, 3)
    print(raw.shape)
    return raw
# r = showImageFromRaw("/tmp/out2.bin")
r2 = showImageFromRaw("/tmp/out.bin")
imshow(r2)

In [ ]:
imshow(np.asarray(Image.open("../../jpeg_tutorial/data/house.jpg")))

In [ ]:
imgarray = np.asarray(img)

In [ ]:
imgarray.shape

In [ ]:
imgarray

In [ ]:
ROOT_DIR="/mnt/optane-ssd/lipeng/imagenet/"
def get_filenames():
    with open("/mnt/optane-ssd/lipeng/imagenet/train.txt", 'r') as f:
        lines = f.readlines()
    fnames = []
    for l in lines:
        fnames.append(l.strip().split()[0])
    return fnames

fnames = get_filenames()

In [ ]:
fnames[:10]

In [ ]:
import subprocess
def get_blocks(fname):
    result = subprocess.run(
                    ['/home/lwangay/workspace/FastGC-icecake/scripts/get_block_offset.bin', fname, "/home/lwangay/workspace/FastGC-icecake/scripts/image_block_stat.txt"], stdout=subprocess.PIPE)
    print(result)

get_blocks(ROOT_DIR+fnames[0])

In [ ]:
import random
random.shuffle(fnames)

In [ ]:
for f in fnames[:20]:
    get_blocks(ROOT_DIR+f)

In [ ]:
with open('image_block_stat.txt', 'r') as f:
    lines = f.readlines()
lines[:10]

In [ ]:
image_stat = []
block_diff = []
count = -1
for l in lines:
    if l.startswith('==='):
        count +=1
        image_stat.append([])
        block_diff.append([])
        continue
    if l.startswith("/mnt"):
        continue
    if(l.startswith("\n")):
        continue
    units= l.rstrip().split(" ")
    # print(units)
    dc_off = int(units[3])*8 + int(units[4][:-1])
    ac_off = int(units[6])*8 + int(units[7])
    # print(dc_off, ac_off)
    # break
    image_stat[count].append({'dc':dc_off, 'ac':ac_off})
    if len(image_stat[count]) > 1:
        block_diff[count].append(image_stat[count][-1]['dc'] - image_stat[count][-2]['dc'])

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
length=6
width = 4
plt.figure(figsize=(length,4))
ax = plt.subplot(111)
linestyles=['-','o-','v-','-s', 'p-','*-','d-','x','D','d','|','--','-.','.-',',-', '-o']
linelables = ['1']
node_list = [32, 64, 96, 128, 160]

#x-axis间距
# ax.xaxis.set_major_locator(MultipleLocator(5))
xs = np.arange(0, len(block_diff[0]), 1)
ax.plot(xs, block_diff[0],linestyles[0], linewidth=1.5, color='C'+str(0), alpha=0.9)
ax.legend(loc=0)
# ax.set_title('(b) rgg')
ax.grid(True)
# plt.locator_params(axis='x', nbins=15)
plt.tight_layout()
plt.show()

In [ ]:
length=6
width = 4
plt.figure(figsize=(length,4))
ax = plt.subplot(111)
linestyles=['-','o-','v-','-s', 'p-','*-','d-','x','D','d','|','--','-.','.-',',-', '-o']
linelables = ['1']
node_list = [32, 64, 96, 128, 160]
idx = 1
#x-axis间距
# ax.xaxis.set_major_locator(MultipleLocator(5))
xs = np.arange(0, len(block_diff[idx]), 1)
ax.plot(xs, block_diff[idx],linestyles[0], linewidth=1.5, color='C'+str(0), alpha=0.9)
ax.legend(loc=0)
# ax.set_title('(b) rgg')
ax.grid(True)
# plt.locator_params(axis='x', nbins=15)
plt.tight_layout()
plt.show()

In [ ]:
length=6
width = 4
plt.figure(figsize=(length,4))
ax = plt.subplot(111)
linestyles=['-','o-','v-','-s', 'p-','*-','d-','x','D','d','|','--','-.','.-',',-', '-o']
linelables = ['1']
node_list = [32, 64, 96, 128, 160]
idx = 2
#x-axis间距
# ax.xaxis.set_major_locator(MultipleLocator(5))
xs = np.arange(0, len(block_diff[idx]), 1)
ax.plot(xs, block_diff[idx],linestyles[0], linewidth=1.5, color='C'+str(0), alpha=0.9)
ax.legend(loc=0)
# ax.set_title('(b) rgg')
ax.grid(True)
# plt.locator_params(axis='x', nbins=15)
plt.tight_layout()
plt.show()

In [ ]:
length=6
width = 4
plt.figure(figsize=(length,4))
ax = plt.subplot(111)
linestyles=['-','o-','v-','-s', 'p-','*-','d-','x','D','d','|','--','-.','.-',',-', '-o']
linelables = ['1']
node_list = [32, 64, 96, 128, 160]
idx = 3
#x-axis间距
# ax.xaxis.set_major_locator(MultipleLocator(5))
xs = np.arange(0, len(block_diff[idx]), 1)
ax.plot(xs, block_diff[idx],linestyles[0], linewidth=1.5, color='C'+str(0), alpha=0.9)
ax.legend(loc=0)
# ax.set_title('(b) rgg')
ax.grid(True)
# plt.locator_params(axis='x', nbins=15)
plt.tight_layout()
plt.show()

In [ ]:
length=6
width = 4
plt.figure(figsize=(length,4))
ax = plt.subplot(111)
linestyles=['-','o-','v-','-s', 'p-','*-','d-','x','D','d','|','--','-.','.-',',-', '-o']
linelables = ['1']
node_list = [32, 64, 96, 128, 160]
idx = 4
#x-axis间距
# ax.xaxis.set_major_locator(MultipleLocator(5))
xs = np.arange(0, len(block_diff[idx]), 1)
ax.plot(xs, block_diff[idx],linestyles[0], linewidth=1.5, color='C'+str(0), alpha=0.9)
ax.legend(loc=0)
# ax.set_title('(b) rgg')
ax.grid(True)
# plt.locator_params(axis='x', nbins=15)
plt.tight_layout()
plt.show()

In [ ]:
length=6
width = 4
plt.figure(figsize=(length,4))
ax = plt.subplot(111)
linestyles=['-','o-','v-','-s', 'p-','*-','d-','x','D','d','|','--','-.','.-',',-', '-o']
linelables = ['1']
node_list = [32, 64, 96, 128, 160]
idx = 5
#x-axis间距
# ax.xaxis.set_major_locator(MultipleLocator(5))
xs = np.arange(0, len(block_diff[idx]), 1)
ax.plot(xs, block_diff[idx],linestyles[0], linewidth=1.5, color='C'+str(0), alpha=0.9)
ax.legend(loc=0)
# ax.set_title('(b) rgg')
ax.grid(True)
# plt.locator_params(axis='x', nbins=15)
plt.tight_layout()
plt.show()

In [ ]:
length=6
width = 4
plt.figure(figsize=(length,4))
ax = plt.subplot(111)
linestyles=['-','o-','v-','-s', 'p-','*-','d-','x','D','d','|','--','-.','.-',',-', '-o']
linelables = ['1']
node_list = [32, 64, 96, 128, 160]
idx = 6
#x-axis间距
# ax.xaxis.set_major_locator(MultipleLocator(5))
xs = np.arange(0, len(block_diff[idx]), 1)
ax.plot(xs, block_diff[idx],linestyles[0], linewidth=1.5, color='C'+str(0), alpha=0.9)
ax.legend(loc=0)

# ax.set_title('(b) rgg')
ax.grid(True)
# plt.locator_params(axis='x', nbins=15)
plt.tight_layout()
plt.show()


In [ ]:
length=6
width = 4
plt.figure(figsize=(length,4))
ax = plt.subplot(111)
linestyles=['-','o-','v-','-s', 'p-','*-','d-','x','D','d','|','--','-.','.-',',-', '-o']
linelables = ['1']
node_list = [32, 64, 96, 128, 160]
idx = 7
#x-axis间距
# ax.xaxis.set_major_locator(MultipleLocator(5))
xs = np.arange(0, len(block_diff[idx]), 1)
ax.plot(xs, block_diff[idx],linestyles[0], linewidth=1.5, color='C'+str(0), alpha=0.9)
ax.legend(loc=0)
# ax.set_title('(b) rgg')
ax.grid(True)
# plt.locator_params(axis='x', nbins=15)
plt.tight_layout()
plt.show()